In [8]:
import pandas as pd # ca on va devoir bcp l'uiliser
import numpy as np
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_rows', None)
from tqdm import tqdm
import os
import glob
from collections import Counter

In [15]:
df = pd.read_csv('/Users/edouard/Desktop/EA p1  HFT/HFT_QR_RL_save/Sans titre/HFT_QR_RL/data/MBO-10 analyse_2/CSV_dezippe_nasdaq/xnas-itch-20240927.mbp-10.csv')
df.head()
#df = df[df['action'] == 'T']


,ts_recv,ts_event,rtype,publisher_id,instrument_id,action,side,depth,price,size,flags,ts_in_delta,sequence,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_ct_00,ask_ct_00,bid_px_01,ask_px_01,bid_sz_01,ask_sz_01,bid_ct_01,ask_ct_01,bid_px_02,ask_px_02,bid_sz_02,ask_sz_02,bid_ct_02,ask_ct_02,bid_px_03,ask_px_03,bid_sz_03,ask_sz_03,bid_ct_03,ask_ct_03,bid_px_04,ask_px_04,bid_sz_04,ask_sz_04,bid_ct_04,ask_ct_04,bid_px_05,ask_px_05,bid_sz_05,ask_sz_05,bid_ct_05,ask_ct_05,bid_px_06,ask_px_06,bid_sz_06,ask_sz_06,bid_ct_06,ask_ct_06,bid_px_07,ask_px_07,bid_sz_07,ask_sz_07,bid_ct_07,ask_ct_07,bid_px_08,ask_px_08,bid_sz_08,ask_sz_08,bid_ct_08,ask_ct_08,bid_px_09,ask_px_09,bid_sz_09,ask_sz_09,bid_ct_09,ask_ct_09,symbol
0,2024-09-27 08:00:00.057997864+00:00,2024-09-27 08:00:00.057831800+00:00,10,2,7152,A,N,0,162.15,35,130,166064,275151,162.15,NaN,35,0,1,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,GOOGL
1,2024-09-27 08:00:00.083821435+00:00,2024-09-27 08:00:00.083555205+00:00,10,2,7152,A,B,1,161.89,100,130,266230,275813,162.15,NaN,35,0,1,0,161.89,NaN,100,0,1,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,GOOGL
2,2024-09-27 08:00:00.083868155+00:00,2024-09-27 08:00:00.083616928+00:00,10,2,7152,A,A,0,162.29,100,128,251227,275825,162.15,162.29,35,100,1,1,161.89,NaN,100,0,1,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,GOOGL
3,2024-09-27 08:00:00.086000347+00:00,2024-09-27 08:00:00.085833622+00:00,10,2,9382,A,N,0,34.60,100,130,166725,276292,34.60,NaN,100,0,1,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,KHC
4,2024-09-27 08:00:00.086016088+00:00,2024-09-27 08:00:00.085848562+00:00,10,2,9382,A,A,0,34.98,100,130,167526,276293,34.60,34.98,100,100,1,1,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,KHC


In [10]:
df = df.replace(614, 0)
depth = 0
df = df[df['symbol'] == 'GOOGL']
df['ts_event'] = pd.to_datetime(df['ts_event'])
df[f'bid_sz_0{depth}_diff'] = df[f'bid_sz_0{depth}'].diff()
df[f'ask_sz_0{depth}_diff'] = df[f'ask_sz_0{depth}'].diff()

df = df[df['depth'] == depth]
#df['action'].value_counts()
# df_ = df[['action','side','size','bid_sz_00','ask_sz_00']]

# df_.head(1)


In [11]:

condition_T = (
    (df['action'] == 'T') &
    (
        ((df['side'] == 'B') & (df[f'bid_sz_0{depth}_diff'] == -df['size'])) |
        ((df['side'] == 'A') & (df[f'ask_sz_0{depth}_diff'] == -df['size']))
    )
)

# Condition pour 'A'
condition_A = (
    (df['action'] == 'A') &
    (
        ((df['side'] == 'B') & (df[f'bid_sz_0{depth}_diff'] == df['size'])) |
        ((df['side'] == 'A') & (df[f'ask_sz_0{depth}_diff'] == df['size']))
    )
)

# Condition pour 'C'
condition_C = (
    (df['action'] == 'C') &
    (
        ((df['side'] == 'B') & (df[f'bid_sz_0{depth}_diff'] == -df['size'])) |
        ((df['side'] == 'A') & (df[f'ask_sz_0{depth}_diff'] == -df['size']))
    )
)

# Appliquer 'OK' ou 'NOK' en fonction des conditions respectées
df['status'] = np.where(condition_T | condition_A | condition_C, 'OK', 'NOK')
#df = df['status' == 'OK']
df['status_N'] = np.where(condition_T | condition_A | condition_C, 3, 2)
df['status_diff'] = df['status_N'].diff()
df_ = df[['action','side','size','bid_sz_00','ask_sz_00','status']]#,'status_N','status_diff']]
df_.head(100)


,action,side,size,bid_sz_00,ask_sz_00,status
0,A,N,35,35,0,NOK
2,A,A,100,35,100,OK
14,C,A,100,35,0,OK
16,A,A,35,35,35,OK
23,C,A,35,35,0,OK
24,A,A,100,35,100,OK
25,A,A,100,35,100,NOK
27,A,A,100,35,100,NOK
29,A,A,100,35,100,NOK
33,C,A,100,35,0,OK


In [ ]:
df_.head(10)

In [ ]:
df_ = df_[df_['status_diff'] != 1 | -1]
df_.head(10)

In [12]:
df_['status'].value_counts()

status
OK     661721
NOK    226924
Name: count, dtype: int64

In [ ]:
df['size'].value_counts()

In [ ]:
def process_dataframe(df):
    # Transformer chaque valeur qui est 614 par 0
    df = df.replace(614, 0)

    # Initialiser une liste pour collecter les DataFrames filtrés
    final_dfs = []

    # 1) Récupérer les valeurs uniques de 'symbol'
    symbols = df['symbol'].unique()
    for symbol in symbols:
        # Filtrer le DataFrame pour chaque 'symbol'
        df_symbol = df[df['symbol'] == symbol]

        # 2) Récupérer les valeurs uniques de 'depth' pour ce 'symbol'
        depths = df_symbol['depth'].unique()

        for depth in depths:
            # Filtrer le DataFrame pour chaque 'depth'
            df_depth = df_symbol[df_symbol['depth'] == depth].copy()

            # Calculer les différences pour 'bid_sz_0x' et 'ask_sz_0x'
            df_depth[f'bid_sz_0{depth}_diff'] = df_depth[f'bid_sz_0{depth}'].diff()
            df_depth[f'ask_sz_0{depth}_diff'] = df_depth[f'ask_sz_0{depth}'].diff()

            # 3) Appliquer les filtres spécifiques donnés
            df_depth = df_depth[~(
                (df_depth['action'] == 'T') &
                (
                    ((df_depth['side'] == 'B') & (df_depth[f'bid_sz_0{depth}_diff'] != -df_depth['size'])) |
                    ((df_depth['side'] == 'A') & (df_depth[f'ask_sz_0{depth}_diff'] != -df_depth['size']))
                )
            )]

            df_depth = df_depth[~(
                (df_depth['action'] == 'A') &
                (
                    ((df_depth['side'] == 'B') & (df_depth[f'bid_sz_0{depth}_diff'] != df_depth['size'])) |
                    ((df_depth['side'] == 'A') & (df_depth[f'ask_sz_0{depth}_diff'] != df_depth['size']))
                )
            )]

            df_depth = df_depth[~(
                (df_depth['action'] == 'C') &
                (
                    ((df_depth['side'] == 'B') & (df_depth[f'bid_sz_0{depth}_diff'] != -df_depth['size'])) |
                    ((df_depth['side'] == 'A') & (df_depth[f'ask_sz_0{depth}_diff'] != -df_depth['size']))
                )
            )]

            # Ajouter le DataFrame filtré à la liste
            final_dfs.append(df_depth)

    # 4) Concaténer tous les DataFrames obtenus pour chaque depth et symbol
    df_final = pd.concat(final_dfs, ignore_index=True)
    symbols = df_final['symbol'].unique()
    # 5) Trier par 'ts_event' qui est une date
    df_final = df_final[df_final['side'].isin(['B','A'])]
    df_final = df_final.sort_values(by='ts_event').reset_index(drop=True)

    # 6) Réindexer le DataFrame et ajouter une colonne 'diff_reindex'
    df_final['reindex'] = df_final.index
    

    # 7) Sélectionner uniquement les symboles qui sont 'GOOGL'
    #df_result = df_final[df_final['symbol'] == 'GOOGL']
    df_result = df_final
    df_result['diff_reindex'] = df_result['reindex'].diff().fillna(0).astype(int)
    #df_result = df_result[df_result['depth'] == 0]
    
    df_result['ts_event'] = pd.to_datetime(df_result['ts_event'])
    df_result['temps_ecoule'] = df_result['ts_event'].diff()
    df_result['temps_ecoule_secondes'] = df_result['temps_ecoule'].dt.total_seconds()
    # Retourner le DataFrame final filtré
    return df_result

# Exemple d'utilisation
# df = pd.read_csv('votre_fichier.csv')
df_result = process_dataframe(df)
# print(df_result)


In [ ]:
df__ = df_result[df_result['symbol'] == 'KHC']
df__[['action','side','depth','size','bid_sz_00','ask_sz_00','symbol']].head(30)

In [ ]:
np.unique(df_result['symbol'].values)

In [ ]:
df = pd.read_csv('/Users/edouard/Desktop/EA p1  HFT/HFT_QR_RL_save/Sans titre/HFT_QR_RL/data/MBO-10 analyse_2/CSV_dezippe_nasdaq/xnas-itch-20240927.mbp-10.csv')
df['index_col'] = df.index
df = df[df['symbol'] == 'GOOGL']

# Convertir 'ts_event' en datetime
df['ts_event'] = pd.to_datetime(df['ts_event'])

# Calculer la différence entre les indices
df['index_diff'] = df['index_col'].diff()

# Initialiser une nouvelle colonne 'temps_ecoule'
df['temps_ecoule'] = 0

# Pour les lignes avec des indices consécutifs (différence de 1), calculer la différence de temps
mask_consecutif = df['index_diff'] == 1
df.loc[mask_consecutif, 'temps_ecoule'] = df['ts_event'].diff().dt.total_seconds()

# Afficher le DataFrame
print(df[['index_col', 'ts_event', 'index_diff', 'temps_ecoule']])

# df['ts_event'] = pd.to_datetime(df['ts_event'])
# df['temps_ecoule'] = df['ts_event'].diff()
# df['temps_ecoule_secondes'] = df['temps_ecoule'].dt.total_seconds()
# df.head(30)

In [ ]:
#df = df[df['depth'] == 0]
df['bid_sz_00_diff'] = df['bid_sz_00'].diff()
df['ask_sz_00_diff'] = df['ask_sz_00'].diff()
# MBO_filtered_depth_0_ = df[
#     ~(
#         (df['action'] == 'C')&
#         (
#             ((df['side'] == 'B')&(df['bid_sz_00_diff'] == 0)) |
#             ((df['side'] == 'A')&(df['ask_sz_00_diff'] == 0))
#         )
#     )
# ]

# df = df[
#     ~(
#         (df['action'] == 'A')&
#         (
#             ((df['side'] == 'B')&(df['bid_sz_00_diff'] != df['size'])) |
#             ((df['side'] == 'A')&(df['ask_sz_00_diff'] != df['size']))
#         )
#     )
# ]
# df = df[
#     ~(
#         (df['action'] == 'T')&
#         (
#             ((df['side'] == 'B')&(df['bid_sz_00_diff'] != -df['size'])) |
#             ((df['side'] == 'A')&(df['ask_sz_00_diff'] != -df['size']))
#         )
#     )
# ]

df.head(30)

In [ ]:
import polars as pl

# Lire le fichier CSV avec Polars
df = pl.read_csv('/Users/edouard/Desktop/EA p1  HFT/HFT_QR_RL_save/Sans titre/HFT_QR_RL/data/MBO-10 analyse_2/CSV_dezippe_nasdaq/xnas-itch-20240927.mbp-10.csv')

# Ajouter une colonne d'index 'index_col'
df = df.with_columns(pl.arange(0, df.height).alias('index_col'))

# Filtrer le DataFrame pour ne garder que les lignes avec 'symbol' == 'GOOGL'
df = df.filter(pl.col('symbol') == 'GOOGL')

# Convertir 'ts_event' en datetime
df = df.with_columns(
    pl.col('ts_event').str.to_datetime().alias('ts_event')
)

# Calculer la différence des indices
df = df.with_columns(
    pl.col('index_col').diff().alias('index_diff')
)

# Calculer la différence de temps entre événements consécutifs
df = df.with_columns(
    pl.when(pl.col('index_diff') == 1)  # Si les indices sont consécutifs
    .then(pl.col('ts_event').diff().dt.nanoseconds() / 1e9)  # Calculer la différence en secondes
    .otherwise(0)  # Sinon, mettre 0
    .alias('temps_ecoule_secondes')
)

# Supprimer les lignes où l'intervalle de temps est 0
df = df.filter(pl.col('temps_ecoule_secondes') != 0)

# Afficher le DataFrame final
print(df)


In [ ]:
df.replace(614, 0, inplace=True)
df['ts_event'] = pd.to_datetime(df['ts_event'])
df = df[df['side'].isin(['B','A'])]
df['temps_ecoule'] = df['ts_event'].diff()
df['temps_ecoule_secondes'] = df['temps_ecoule'].dt.total_seconds()

# df['ask_size'] = df['ask_sz_00']
# df['bid_size'] = df['ask_sz_00']
# df['bid_sz_00_shifted'] = df['bid_sz_00'].shift(1)
# df['bid_sz_shifted'].iloc[0] = 0

df = df[
    ~(
        (df['action'] == 'A')&
        (
            ((df['side'] == 'B')&(df['bid_sz_00_diff'] != df['size'])) |
            ((df['side'] == 'A')&(df['ask_sz_00_diff'] != df['size']))
        )
    )
]

df = df[
    ~(
        (df['action'] == 'T')&
        (
            ((df['side'] == 'B')&(df['bid_sz_00_diff'] != -df['size'])) |
            ((df['side'] == 'A')&(df['ask_sz_00_diff'] != -df['size']))
        )
    )
]

df = df[
    ~(
        (df['action'] == 'C')&
        (
            ((df['side'] == 'B')&(df['bid_sz_00_diff'] != -df['size'])) |
            ((df['side'] == 'A')&(df['ask_sz_00_diff'] != -df['size']))
        )
    )
]

#df.head(30)
#df = df[df['action'] == 'T']
df_filtered = df[['action','side','size','bid_sz_00','ask_sz_00']]
# # Initialiser la variable pour la ligne précédente à None
# previous_row = None

# # Itérer sur les lignes du DataFrame
# # Supposons que df_filtered est votre DataFrame original
# # Ajoutez des colonnes 'ask_size' et 'bid_size' initialisées à 0
# df_filtered['ask_size'] = 0
# df_filtered['bid_size'] = 0

# Initialiser la variable pour la ligne précédente à None
# previous_row = None

# # Itérer sur les lignes du DataFrame en utilisant iterrows()
# for index, row in tqdm(df_filtered.iterrows()):
#     if previous_row is not None:
#         # Vérifier la valeur de la colonne 'side'
#         if row['side'] == 'A':
#             # Mettre à jour 'ask_size' en fonction de l'action
#             if row['action'] == 'A':
#                 df_filtered.at[index, 'ask_size'] = previous_row['ask_size'] + row['size']
#             elif row['action'] == 'C':
#                 df_filtered.at[index, 'ask_size'] = previous_row['ask_size'] - row['size']
#             elif row['action'] == 'T':
#                 df_filtered.at[index, 'ask_size'] = previous_row['ask_size'] - row['size']
#             # Garder la même valeur pour 'bid_size'
#             df_filtered.at[index, 'bid_size'] = previous_row['bid_size']
            
#         elif row['side'] == 'B':
#             # Mettre à jour 'bid_size' en fonction de l'action
#             if row['action'] == 'A':
#                 df_filtered.at[index, 'bid_size'] = previous_row['bid_size'] + row['size']
#             elif row['action'] == 'C':
#                 df_filtered.at[index, 'bid_size'] = previous_row['bid_size'] - row['size']
#             elif row['action'] == 'T':
#                 df_filtered.at[index, 'bid_size'] = previous_row['bid_size'] - row['size']
#             # Garder la même valeur pour 'ask_size'
#             df_filtered.at[index, 'ask_size'] = previous_row['ask_size']
#     else:
#         # Si c'est la première ligne, initialiser 'ask_size' et 'bid_size' avec la valeur de 'size'
#         if row['side'] == 'A':
#             df_filtered.at[index, 'ask_size'] = row['size']
#         elif row['side'] == 'B':
#             df_filtered.at[index, 'bid_size'] = row['size']

#     # Mettre à jour la ligne précédente
#     previous_row = df_filtered.loc[index]
df_filtered.head(100)


In [16]:
df = pd.read_csv('/Users/edouard/Desktop/EA p1  HFT/HFT_QR_RL_save/Sans titre/HFT_QR_RL/data/MBO-10 analyse_2/CSV_dezippe_nasdaq/xnas-itch-20240927.mbp-10.csv')
df = df[df['symbol'] == 'GOOGL']

In [17]:
price = df[df['action'] =='T']['price'].to_numpy()
time_price = pd.to_datetime(df[df['action'] =='T']['ts_event'])
bid_px_00 = df['bid_px_00'].to_numpy()
ask_px_00 = df['ask_px_00'].to_numpy()
time = pd.to_datetime(df['ts_event'])
len(price)


69782

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y = price, x = time_price, mode ='lines', name ='Trade', showlegend = True))
fig.add_trace(go.Scatter(y = bid_px_00, x = time,  mode ='lines', name ='Bid', showlegend = True))
fig.add_trace(go.Scatter(y = ask_px_00, x = time,  mode ='lines', name ='Ask', showlegend = True))
fig.update_layout(title=f'Trades de GOOGLE+bid_ask', xaxis_title='size', yaxis_title='intensity', showlegend=True)
fig.show()